### Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
import string
from datetime import datetime, date
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)

### Reading Data Sets

In [ ]:
df_train = pd.read_csv('../input/consumer-complaint-resolution-data-set/Consumer_Complaints_train.csv')
df_test = pd.read_csv('../input/consumer-complaint-resolution-data-set/Consumer_Complaints_test.csv')

### Train Data

In [ ]:
df_train.head()

### Test Data

In [ ]:
df_test.head()

### Shape of Train and Test Data

In [ ]:
df_train.shape, df_test.shape

### Columns of Train and Test Data

In [ ]:
df_train.columns

In [ ]:
df_test.columns

### Data Types

In [ ]:
data_types_train = pd.DataFrame(df_train.dtypes, columns = ['Train'])
data_types_test = pd.DataFrame(df_test.dtypes, columns = ['Test'])
data_types = pd.concat([data_types_train, data_types_test], axis = 1)
data_types

### Missing Values

In [ ]:
missing_values_train = pd.DataFrame(df_train.isna().sum(), columns = ['Train'])
missing_values_test = pd.DataFrame(df_test.isna().sum(), columns = ['Test'])
missing_values = pd.concat([missing_values_train, missing_values_test], axis = 1)
missing_values

### Dropping Missing Values

In [ ]:
columns_with_missing_values = ['Sub-product', 'Sub-issue', 'Consumer complaint narrative', 'Company public response', 'Tags', 'Consumer consent provided?']
df_train = df_train.drop(columns_with_missing_values, axis = 1)
df_test = df_test.drop(columns_with_missing_values, axis = 1)

### Extracting Date, Month and Year from Date Received Column

In [ ]:
df_train['Year_Received'] = df_train['Date received'].apply(lambda dateString : datetime.strptime(dateString,'%Y-%m-%d').year)
df_test['Year_Received'] = df_test['Date received'].apply(lambda dateString : datetime.strptime(dateString,'%Y-%m-%d').year)
df_train['Month_Received'] = df_train['Date received'].apply(lambda dateString : datetime.strptime(dateString,'%Y-%m-%d').month)
df_test['Month_Received'] = df_test['Date received'].apply(lambda dateString : datetime.strptime(dateString,'%Y-%m-%d').month)
df_train['Day_Received'] = df_train['Date received'].apply(lambda dateString : datetime.strptime(dateString,'%Y-%m-%d').day)
df_test['Day_Received'] = df_test['Date received'].apply(lambda dateString : datetime.strptime(dateString,'%Y-%m-%d').day)

### Extracting Date, Month and Year From Date Sent to the Company Column

In [ ]:
df_train['Year_Sent'] = df_train['Date sent to company'].apply(lambda dateString : datetime.strptime(dateString,'%Y-%m-%d').year)
df_test['Year_Sent'] = df_test['Date sent to company'].apply(lambda dateString : datetime.strptime(dateString,'%Y-%m-%d').year)
df_train['Month_Sent'] = df_train['Date sent to company'].apply(lambda dateString : datetime.strptime(dateString,'%Y-%m-%d').month)
df_test['Month_Sent'] = df_test['Date sent to company'].apply(lambda dateString : datetime.strptime(dateString,'%Y-%m-%d').month)
df_train['Day_Sent'] = df_train['Date sent to company'].apply(lambda dateString : datetime.strptime(dateString,'%Y-%m-%d').day)
df_test['Day_Sent'] = df_test['Date sent to company'].apply(lambda dateString : datetime.strptime(dateString,'%Y-%m-%d').day)

### Converting Dates from Object Type to Datetime Type

In [ ]:
df_train['Date received'] = pd.to_datetime(df_train['Date received'])
df_test['Date received'] = pd.to_datetime(df_test['Date received'])
df_train['Date sent to company'] = pd.to_datetime(df_train['Date sent to company'])
df_test['Date sent to company'] = pd.to_datetime(df_test['Date sent to company'])

### Calculating the Number of Days the Complaint was with the Company

In [ ]:
df_train['Days held'] = df_train['Date sent to company'] - df_train['Date received']
df_test['Days held'] = df_test['Date sent to company'] - df_test['Date received']

### Converting Days Held to Int

In [ ]:
df_train['Days held'] = df_train['Days held'].astype('timedelta64[D]').astype(int)
df_test['Days held'] = df_test['Days held'].astype('timedelta64[D]').astype(int)

### Dropping Date Received and Date Sent to Company

In [ ]:
df_train = df_train.drop(['Date received', 'Date sent to company'], axis = 1)
df_test = df_test.drop(['Date received', 'Date sent to company'], axis = 1)

### Dropping ZIP Code, Complaint ID

In [ ]:
df_train = df_train.drop(['ZIP code', 'Complaint ID'], axis = 1)
df_test = df_test.drop(['ZIP code', 'Complaint ID'], axis = 1)

### Imputing Nulls in State by Mode

In [ ]:
df_train['State'].mode(), df_test['State'].mode()

In [ ]:
df_train['State'] = df_train['State'].replace(np.nan, 'CA')
df_test['State'] = df_test['State'].replace(np.nan, 'CA')

### Checking Missing Values  Data

In [ ]:
missing_values_train = pd.DataFrame(df_train.isna().sum(), columns = ['Train'])
missing_values_test = pd.DataFrame(df_test.isna().sum(), columns = ['Test'])
missing_values = pd.concat([missing_values_train, missing_values_test], axis = 1)
missing_values

### Dropping Year Sent, Month Sent and Day Sent

In [ ]:
df_train = df_train.drop(['Year_Sent', 'Month_Sent', 'Day_Sent'], axis = 1)
df_test = df_test.drop(['Year_Sent', 'Month_Sent', 'Day_Sent'], axis = 1)

### Categorizing Days into Weeks

In [ ]:
week_train = []
for i in df_train['Day_Received']:
    if i < 8:
        week_train.append(1)
    elif i >= 8 and i < 16:
        week_train.append(2)
    elif i >=16 and i < 22:
        week_train.append(3)
    else:
        week_train.append(4)
df_train['Week_Received'] = week_train
week_test = []
for i in df_test['Day_Received']:
    if i < 8:
        week_test.append(1)
    elif i >= 8 and i < 16:
        week_test.append(2)
    elif i >=16 and i < 22:
        week_test.append(3)
    else:
        week_test.append(4)
df_test['Week_Received'] = week_test

### Dropping Day_Received

In [ ]:
df_train = df_train.drop(['Day_Received'], axis = 1)
df_test = df_test.drop(['Day_Received'], axis = 1)

### Train Data

In [ ]:
df_train.head()

### Test Data

In [ ]:
df_test.head()

### Disputed Consumers Data

In [ ]:
disputed_cons = df_train[df_train['Consumer disputed?'] == 'Yes']

### Number of Disputes

In [ ]:
sns.countplot(x = 'Consumer disputed?', data = df_train)
plt.xticks(fontsize = 10, ha = "right")
plt.yticks(fontsize = 10)
plt.xlabel('Consumer disputed', fontsize = 15)
plt.ylabel('Count', fontsize = 15)

        Roughly 21% of the consumer have disputed.

### Products-wise Disputes

In [ ]:
sns.countplot(x = 'Product', hue = 'Consumer disputed?', data = df_train)
plt.xticks(rotation = 90, fontsize = 10, ha = "right")
plt.yticks(fontsize = 10)
plt.xlabel('Product', fontsize = 15)
plt.ylabel('Count', fontsize = 15)

        Roughly 37% consumer with mortgage have disputed.
        Approx. 54% consumer who have disputed are from mortgage or debt collection.
        68% consumer having disputes are from mortgage, debt collection and credit reporting.
        Adding credit card consumers and bank account or services consumer will make it 80% and 91% respectively.

### Top Issues with Highest Disputes

In [ ]:
top_issues_disputes = disputed_cons['Issue'].value_counts().sort_values(ascending = False).head(10)
sns.barplot(x = top_issues_disputes.index, y = top_issues_disputes.values)
plt.xticks(rotation = 90, fontsize = 10, ha = "right")
plt.yticks(fontsize = 10)
plt.xlabel('Issues', fontsize = 15)
plt.ylabel('Count', fontsize = 15)

        66% disputes are from these 10 issues.

### State with Maximum Disputes

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
sns.countplot(x = df_train['State'])
plt.xticks(rotation = 90, fontsize = 10, ha = "right")
plt.yticks(fontsize = 10)
plt.xlabel('State', fontsize = 15)
plt.ylabel('Count', fontsize = 15)

        15% disputes from CA.
        25% disputes from CA and FL
        32% disputes from CA, FL and TX.
        38% disputes from CA, FL, TX and NY.
        43% disputes from CA, FL, TX, NY and GA.
        47% disputes from CA, FL, TX, NY, GA and NJ.
        50% disputes from CA, FL, TX, NY, GA, NJ and IL.
        54% disputes from CA, FL, TX, NY, GA, NJ, IL and VA.
        57% disputes from CA, FL, TX, NY, GA, NJ, IL, VA and PA.
        61% disputes from CA, FL, TX, NY, GA, NJ, IL, VA, PA and MD.

### Maximum Disputes Submitted Via

In [ ]:
sns.countplot(x = 'Submitted via', data = disputed_cons)
plt.xticks(rotation = 90, fontsize = 10, ha = "right")
plt.yticks(fontsize = 10)
plt.xlabel('Submitted Via', fontsize = 15)
plt.ylabel('Count', fontsize = 15)

        72% disputes are submitted via web.
        88% disputes are submitted via web and referral.

### Company's Response to the Complaints

In [ ]:
sns.countplot(x = 'Company response to consumer', data = df_train)
plt.xticks(rotation = 90, fontsize = 10, ha = "right")
plt.yticks(fontsize = 10)
plt.xlabel('Company Response', fontsize = 15)
plt.ylabel('Count', fontsize = 15)

        74% complaints are closed with explanation.

### Company's Response Leading to Disputes

In [ ]:
sns.countplot(x = 'Company response to consumer', data = disputed_cons)
plt.xticks(rotation = 90, fontsize = 10, ha = "right")
plt.yticks(fontsize = 10)
plt.xlabel('Company Response', fontsize = 15)
plt.ylabel('Count', fontsize = 15)

        82% disputes are closed with explanation at the initial stage.
        89% disputes are either closed with explanation or non-monetary relief in the earlier stage.

### Whether there are Disputes Instead of Timely Response

In [ ]:
sns.countplot(x = 'Timely response?', data = disputed_cons)
plt.xticks(fontsize = 10, ha = "right")
plt.yticks(fontsize = 10)
plt.xlabel('Timely Response', fontsize = 15)
plt.ylabel('Count', fontsize = 15)

        98% disputes were timely repsonded at the intial stages.

### Year Wise Complaints

In [ ]:
sns.countplot(x = 'Year_Received', data = df_train)
plt.xticks(fontsize = 10, ha = "right")
plt.yticks(fontsize = 10)
plt.xlabel('Year', fontsize = 15)
plt.ylabel('Count', fontsize = 15)

        28% complaints are raised in 2015.
        53% complaints are raised in 2014 and 2015.
        71% complaints are raised in 2013 to 2015.
        88% complaints are raised in 2013 to 2016.

### Year Wise Disputes

In [ ]:
sns.countplot(x = 'Year_Received', data = disputed_cons)
plt.xticks(fontsize = 10, ha = "right")
plt.yticks(fontsize = 10)
plt.xlabel('Year', fontsize = 15)
plt.ylabel('Count', fontsize = 15)

        27% disputes are raised in 2015.
        50% disputes are raised in 2014 and 2015.
        69% disputes are raised in 2014 to 2016.
        87% disputes are raised in 2013 to 2016.

### Month Wise Complaints

In [ ]:
sns.countplot(x = 'Month_Received', data = df_train)
plt.xticks(fontsize = 10, ha = "right")
plt.yticks(fontsize = 10)
plt.xlabel('Month', fontsize = 15)
plt.ylabel('Count', fontsize = 15)

        64% complaints are raised in January to July.
        72% complaints are raised in January to August.

### Month Wise Disputes

In [ ]:
sns.countplot(x = 'Month_Received', data = disputed_cons)
plt.xticks(fontsize = 10, ha = "right")
plt.yticks(fontsize = 10)
plt.xlabel('Month', fontsize = 15)
plt.ylabel('Count', fontsize = 15)

        56% disputes are raised in March to August.
        73% disputes are raised in January to August.

### Week Wise Complaints

In [ ]:
sns.countplot(x = 'Week_Received', data = df_train)
plt.xticks(fontsize = 10, ha = "right")
plt.yticks(fontsize = 10)
plt.xlabel('Week', fontsize = 15)
plt.ylabel('Count', fontsize = 15)

        30% complaints are in the 4th week of the month.
        50% complaints are in both the half of the month.

### Week Wise Disputes 

In [ ]:
sns.countplot(x = 'Week_Received', data = disputed_cons)
plt.xticks(fontsize = 10, ha = "right")
plt.yticks(fontsize = 10)
plt.xlabel('Week', fontsize = 15)
plt.ylabel('Count', fontsize = 15)

        30% disputes are in the 4th week of the month.
        50% disputes are in both the half of the month.

### Top Companies with Highest Complaints

In [ ]:
worst_company_complaints = df_train['Company'].value_counts().sort_values(ascending = False).head(10)
sns.barplot(x = worst_company_complaints.index, y = worst_company_complaints.values)
plt.xticks(rotation = 90, fontsize = 10, ha = "right")
plt.yticks(fontsize = 10)
plt.xlabel('Company', fontsize = 15)
plt.ylabel('Count', fontsize = 15)

        53% complaints are for these 10 companies.

### Top Companies with Highest Disputes

In [ ]:
worst_company_disputes = disputed_cons['Company'].value_counts().sort_values(ascending = False).head(10)
sns.barplot(x = worst_company_complaints.index, y = worst_company_complaints.values)
plt.xticks(rotation = 90, fontsize = 10, ha = "right")
plt.yticks(fontsize = 10)
plt.xlabel('Company', fontsize = 15)
plt.ylabel('Count', fontsize = 15)

        53% disputes are for these 10 companies.

### Days Held Column Analysis

In [ ]:
df_train['Days held'].describe()

In [ ]:
df_test['Days held'].describe()

### Converting Negative Days Held to Zero

In [ ]:
Days_held_train = []
for i in df_train['Days held']:
    if i < 0:
        Days_held_train.append(0)
    else:
        Days_held_train.append(i)
df_train['Days_held'] = Days_held_train
Days_held_test = []
for i in df_test['Days held']:
    if i < 0:
        Days_held_test.append(0)
    else:
        Days_held_test.append(i)
df_test['Days_held'] = Days_held_test

### Dropping Days Held with Negative Values


In [ ]:
df_train = df_train.drop('Days held', axis = 1)
df_test = df_test.drop('Days held', axis = 1)

### NLP Pre-Processing

In [ ]:
relevant_text_train = df_train['Issue']
relevant_text_test = df_test['Issue']
tokenized_data_train = relevant_text_train.apply(lambda x: wordpunct_tokenize(x.lower()))
tokenized_data_test = relevant_text_test.apply(lambda x: wordpunct_tokenize(x.lower()))
def remove_punctuation(text):
    no_punctuation = []
    for w in text:
        if w not in string.punctuation:
            no_punctuation.append(w)
    return no_punctuation
no_punctuation_data_train = tokenized_data_train.apply(lambda x: remove_punctuation(x))
no_punctuation_data_test = tokenized_data_test.apply(lambda x: remove_punctuation(x))
stop_words = stopwords.words('english')
filtered_sentence_train = [w for w in no_punctuation_data_train if not w in stop_words]
filtered_sentence_train = pd.Series(filtered_sentence_train)
filtered_sentence_test = [w for w in no_punctuation_data_test if not w in stop_words]
filtered_sentence_test = pd.Series(filtered_sentence_test)
def lemmatize_text(text):
    lem_text = [WordNetLemmatizer().lemmatize(w,pos = 'v') for w in text]
    return lem_text
lemmatized_data_train = filtered_sentence_train.apply(lambda x:lemmatize_text(x))
lemmatized_data_test = filtered_sentence_test.apply(lambda x:lemmatize_text(x))
def stem_text(text):
    stem_text = [PorterStemmer().stem(w) for w in text]
    return stem_text
stemmed_data_train = lemmatized_data_train.apply(lambda x:stem_text(x))
stemmed_data_test = lemmatized_data_test.apply(lambda x:stem_text(x))
def word_to_sentence(text):
    text_sentence = " ".join(text)
    return text_sentence
clean_data_train = stemmed_data_train.apply(lambda x:word_to_sentence(x))
clean_data_test = stemmed_data_test.apply(lambda x:word_to_sentence(x))

### Concating Clean Data to Train and Test Data and Dropping Issue

In [ ]:
df_train['Issues_cleaned'] = clean_data_train
df_test['Issues_cleaned'] = clean_data_test
df_train = df_train.drop('Issue', axis = 1)
df_test = df_test.drop('Issue', axis = 1)

### Dropping Unneccesary Columns for the Model Building

In [ ]:
drop_cols = ['Company', 'State', 'Year_Received', 'Days_held']
df_train = df_train.drop(drop_cols, axis = 1)
df_test = df_test.drop(drop_cols, axis = 1)

### Changing Consumer Disputed Column to 0 and 1

In [ ]:
df_train['Consumer disputed?'] = np.where(df_train['Consumer disputed?'] == "Yes", 1, 0)

### Creating Dummy Variables

In [ ]:
dum_cols = ['Product', 'Submitted via', 'Company response to consumer', 'Timely response?']
df_train_dummies = pd.get_dummies(df_train[dum_cols], prefix_sep = '_', drop_first = True)
df_test_dummies = pd.get_dummies(df_test[dum_cols], prefix_sep = '_', drop_first = True)

### Concating Dummy Variables and Dropping the Original Columns

In [ ]:
df_train = df_train.drop(dum_cols, axis = 1)
df_test = df_test.drop(dum_cols, axis = 1)
df_train = pd.concat([df_train, df_train_dummies], axis = 1)
df_test = pd.concat([df_test, df_test_dummies], axis = 1)

### Calculating TF-IDF

In [ ]:
tf = TfidfVectorizer()
issues_cleaned_train = tf.fit_transform(df_train['Issues_cleaned']).toarray()
issues_cleaned_test = tf.fit_transform(df_test['Issues_cleaned']).toarray()
tf_columns_train = []
tf_columns_test = []
for i in range(issues_cleaned_train.shape[1]):
    tf_columns_train.append('Feature' + str(i+1))
for i in range(issues_cleaned_test.shape[1]):
    tf_columns_test.append('Feature' + str(i+1))
issues_train = pd.DataFrame(issues_cleaned_train, columns = tf_columns_train)
issues_test = pd.DataFrame(issues_cleaned_test, columns = tf_columns_test)
weights = pd.DataFrame(tf.idf_, index = tf.get_feature_names(), columns = ['Idf_weights']).sort_values(by = 'Idf_weights', ascending = False)
weights.head()

### Replacing Issues_cleaned by Vectorized Issues

In [ ]:
df_train = df_train.drop('Issues_cleaned', axis = 1)
df_test = df_test.drop('Issues_cleaned', axis = 1)
df_train = pd.concat([df_train, issues_train], axis = 1)
df_test = pd.concat([df_test, issues_test], axis = 1)
Feature168 = [0] * 119606
df_test['Feature168'] = Feature168

### Train Data

In [ ]:
df_train.head()

### Test Data

In [ ]:
df_test.head()

### Shape of Train and Test Data

In [ ]:
df_train.shape, df_test.shape

### Scaling the Data Sets

In [ ]:
df_train_scaled = pd.DataFrame(StandardScaler().fit_transform(df_train.drop('Consumer disputed?', axis = 1)), columns = df_test.columns)
df_test_scaled = pd.DataFrame(StandardScaler().fit_transform(df_test), columns = df_test.columns)

### Feature Selection

In [ ]:
pca_columns = []
for i in range(df_train_scaled.shape[1]):
    pca_columns.append('PC' + str(i+1))
pca_model = PCA()
pca_model.fit(df_train_scaled)
df_pca_train = pd.DataFrame(pca_model.transform(df_train_scaled), columns = pca_columns)
explained_info_train = pd.DataFrame(pca_model.explained_variance_ratio_, columns=['Explained Info']).sort_values(by = 'Explained Info', ascending = False)
imp = []
for i in range(explained_info_train.shape[0]):
    imp.append(explained_info_train.head(i).sum())
explained_info_train_sum = pd.DataFrame()
explained_info_train_sum['Variable'] = pca_columns
explained_info_train_sum['Importance'] = imp
explained_info_train_sum.head(60)

        So 53 variables are making upto 80% of the information.

In [ ]:
pca_columns = []
for i in range(53):
    pca_columns.append('PC' + str(i+1))
pca_model = PCA(n_components = 53)
pca_model.fit(df_train_scaled)
df_pca_train = pd.DataFrame(pca_model.transform(df_train_scaled), columns = pca_columns)

In [ ]:
df_pca_train.head()

In [ ]:
pca_model = PCA(n_components = 53)
pca_model.fit(df_test_scaled)
df_pca_test = pd.DataFrame(pca_model.transform(df_test_scaled), columns = pca_columns)

### Spliting the Data Sets Into X and Y

In [ ]:
X = df_pca_train
y = df_train['Consumer disputed?']

### Train, Validation and Test Data Sets

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state = 17)
X_test = df_pca_test

### Shapes of the Data Sets

In [ ]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape, X_test.shape

### Performance of Different Algorithms

In [ ]:
models = [LogisticRegression(), DecisionTreeClassifier(), RandomForestClassifier(), AdaBoostClassifier(), GradientBoostingClassifier(), KNeighborsClassifier(), XGBClassifier()]
model_names = ['LogisticRegression', 'DecisionTreeClassifier', 'RandomForestClassifier', 'AdaBoostClassifier', 'GradientBoostingClassifier', 'KNeighborsClassifier', 'XGBClassifier']
accuracy_train = []
accuracy_val = []
for model in models:
    mod = model
    mod.fit(X_train, y_train)
    y_pred_train = mod.predict(X_train)
    y_pred_val = mod.predict(X_val)
    accuracy_train.append(accuracy_score(y_train, y_pred_train))
    accuracy_val.append(accuracy_score(y_val, y_pred_val))
data = {'Modelling Algorithm' : model_names, 'Train Accuracy' : accuracy_train, 'Validation Accuracy' : accuracy_val}
data = pd.DataFrame(data)
data['Difference'] = ((np.abs(data['Train Accuracy'] - data['Validation Accuracy'])) * 100)/(data['Train Accuracy'])
data.sort_values(by = 'Validation Accuracy', ascending = False)

        LogisticRegression is the best model to build the model.

### Final Model and Prediction

In [ ]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred_test = lr.predict(X_test)
y_pred_test = pd.DataFrame(y_pred_test, columns = ['Prediction'])
y_pred_test.head()

### Exporting Predictions to CSV

In [ ]:
y_pred_test.to_csv('Prediction.csv')